<a href="https://colab.research.google.com/github/MoralesCMaJo/Tarea-2--An-lisis-de-series-temporales/blob/main/11_auto_arima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Packages

In [ ]:
!pip install arch
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.7/981.7 kB 16.7 MB/s eta 0:00:00


Loading the data

In [ ]:
raw_data = yfinance.download(tickers="^GSPC ^FTSE ^N225 ^GDAXI", start="1994-01-07", end="2023-09-22", interval="1d", group_by='ticker', auto_adjust=True, threads=True)

[*********************100%%**********************]  4 of 4 completed


In [ ]:
df_comp = raw_data.copy()


In [ ]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

Creating Returns

In [ ]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

Splitting the Data

In [ ]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

Fitting a Model

In [ ]:
!pip install pmdarima
from pmdarima.arima import auto_arima
model_auto = auto_arima(df.ret_ftse[1:])
model_auto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.5 MB/s eta 0:00:00


ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True)

In [ ]:
model_auto.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 6198
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -9467.424
Date:                Sat, 14 Oct 2023   AIC                          18956.848
Time:                        06:32:25   BIC                          19030.900
Sample:                    01-11-1994   HQIC                         18982.522
                         - 10-12-2017                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0266      0.017      1.534      0.125      -0.007       0.061
ar.L1          0.0954      0.077      1.232      0.218      -0.056       0.247
ar.L2         -0.5629      0.077     -7.287      0.000      -0.714      -0.411
ar.L3         -0.1137      0.071     -1.608      0.108      -0.252       0.025
ar.L4          0.2900      0.074      3.898      0.000       0.144       0.436
ma.L1         -0.1153      0.077     -1.490      0.136      -0.267       0.036
ma.L2          0.5215      0.079      6.619      0.000       0.367       0.676
ma.L3          0.0358      0.070      0.513      0.608      -0.101       0.173
ma.L4         -0.2879      0.075     -3.860      0.000      -0.434      -0.142
ma.L5         -0.0897      0.008    -10.589      0.000      -0.106      -0.073
sigma2         1.2417      0.012    102.841      0.000       1.218       1.265
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              8499.29
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                            -0.20
Prob(H) (two-sided):                  0.01   Kurtosis:                         8.72
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Important Arguments

In [ ]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))

In [ ]:
model_auto.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                        
==============================================================================================
Dep. Variable:                                      y   No. Observations:                 7748
Model:             SARIMAX(3, 0, 3)x(0, 0, [1, 2], 5)   Log Likelihood              -11753.349
Date:                                Sat, 14 Oct 2023   AIC                          23528.699
Time:                                        06:50:54   BIC                          23605.206
Sample:                                    01-11-1994   HQIC                         23554.928
                                         - 09-21-2023                                         
Covariance Type:                                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0258      0.027      0.945      0.345      -0.028       0.079
drift      -1.871e-06   6.25e-06     -0.300      0.765   -1.41e-05    1.04e-05
ar.L1         -0.3021      0.071     -4.235      0.000      -0.442      -0.162
ar.L2         -0.2584      0.071     -3.642      0.000      -0.397      -0.119
ar.L3          0.3822      0.073      5.270      0.000       0.240       0.524
ma.L1          0.2821      0.068      4.128      0.000       0.148       0.416
ma.L2          0.2149      0.070      3.090      0.002       0.079       0.351
ma.L3         -0.4610      0.070     -6.627      0.000      -0.597      -0.325
ma.S.L5       -0.0353      0.007     -4.791      0.000      -0.050      -0.021
ma.S.L10      -0.0114      0.007     -1.537      0.124      -0.026       0.003
sigma2         1.2196      0.009    129.151      0.000       1.201       1.238
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):             18520.43
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.76   Skew:                            -0.32
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.55
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""